In [3]:
import cv2
import os
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

%pylab inline 
from IPython.display import clear_output
import time
import PIL.Image
from io import BytesIO
import IPython.display
from mtcnn.mtcnn import MTCNN
import keras

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
NOTEBOOK_DIR = !pwd
NOTEBOOK_DIR = NOTEBOOK_DIR[0]
ROOT_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir))
DATA_DIR = os.path.join(ROOT_DIR, 'data')

MNM_MODEL_DIR = os.path.join(ROOT_DIR, 'models/keras_sequential_model')

In [ ]:
#Use 'jpeg' instead of 'png' (~5 times faster)
def array_to_image(a, fmt='jpeg'):
    #Create binary stream object
    f = BytesIO()

    #Convert array to binary stream object
    PIL.Image.fromarray(a).save(f, fmt)

    return IPython.display.Image(data=f.getvalue())

def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()

    #flip image for natural viewing
    frame = cv2.flip(frame, 1)

    return frame

In [ ]:
# cam = cv2.VideoCapture(0)

# d = IPython.display.display("", display_id=1)
# d2 = IPython.display.display("", display_id=2)
# while True:
#     try:
#         t1 = time.time()
#         frame = get_frame(cam)
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#         im = array_to_image(frame)

#         d.update(im)

#         t2 = time.time()

#         s = f"""{int(1/(t2-t1))} FPS"""
#         d2.update( IPython.display.HTML(s) )
#     except KeyboardInterrupt:
#         print()
#         cam.release()
#         IPython.display.clear_output()
#         print ("Stream stopped")
#         break


In [2]:
model = keras.models.load_model("../models/example_model")
detector = MTCNN()
cap = cv2.VideoCapture("/Users/brtonnies/Desktop/Screen Recording 2020-11-04 at 5.55.09 PM.mov")
rval, frame = cap.read()

ret, frame = cap.read()

count = 1
while cap.isOpened(): 
    #Capture frame-by-frame
#     ret, frame = cap.read()
#     frame = get_frame(cap)

    if frame is not None:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image = array_to_image(frame)

        #Use MTCNN to detect faces
        faces = detector.detect_faces(frame)
        if faces != []:
            
            data = list()
            results = list()
            for face in faces:
                bounding_box = face['box']
                keypoints = face['keypoints']
                data.append([count, bounding_box])
                
                x, y, w, h = bounding_box
                img = frame[y:y+h,x:x+w]
                if img != []:
                    new_img = cv2.resize(img,(50,50))
                    new_img = new_img.reshape(-1,50,50,1)
                    prediction = model.predict(new_img)
                    print("PREDICTION: {}".format(prediction))
                    results.append([count, bounding_box, prediction])
                    cv2.rectangle(frame,
                              (x, y),
                              (x+w, y + h),
                              (0,155,255),
                              2)

                    cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
                    cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
                    cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
                    cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
                    cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)

                
            
#             for (x, y, w, h) in [face['box'] for face in faces]:
#                 img = frame[y:y+h,x:x+w]
#                 img = img[bounding_box[1][1]:j[1][1]+j[1][3],j[1][0]:j[1][0]+j[1][2]]
#                 new_img = cv2.resize(img,(50,50))
#                 new_img = new_img.reshape(-1,50,50,1)
#                 prediction = model.predict(new_img)

#                 bounding_box = face['box']
#                 keypoints = face['keypoints']

        #display resulting frame
        cv2.imshow('frame', frame)
        print(frame.shape)
        

    ret, frame = cap.read()
    clear_output(wait=True)
    count += 1
    if cv2.waitKey(1) &0xFF == ord('q'):
        break
#When everything's done, release capture
cap.release()
cv2.destroyAllWindows()

NameError: name 'keras' is not defined

In [ ]:
cap.release()
cv2.destroyAllWindows()